In [0]:
delta_path = "/Volumes/workspace/ecommerce/ecommerce_data/delta/events"

from pyspark.sql.functions import lit, col

events = spark.read.format("delta").load(delta_path)

incremental_events = events.limit(100) \
    .withColumn("price", col("price") + 10)


In [0]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, delta_path)

merge_condition = """
    t.product_id = s.product_id AND
    t.user_id = s.user_id AND
    t.event_time = s.event_time
"""

delta_table.alias("t").merge(
    incremental_events.alias("s"),
    merge_condition
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()


In [0]:
%sql
DESCRIBE HISTORY delta.`/Volumes/workspace/ecommerce/ecommerce_data/delta/events`;


In [0]:
spark.read.format("delta") \
    .option("versionAsOf", 0) \
    .load(delta_path) \
    .show()


In [0]:
spark.read.format("delta") \
    .option("timestampAsOf", "2026-01-12 10:00:00") \
    .load(delta_path)


In [0]:

%sql
OPTIMIZE delta.`/Volumes/workspace/ecommerce/ecommerce_data/delta/events`;


In [0]:
%sql
OPTIMIZE delta.`/Volumes/workspace/ecommerce/ecommerce_data/delta/events`
ZORDER BY (product_id, user_id);


In [0]:
%sql
VACUUM delta.`/Volumes/workspace/ecommerce/ecommerce_data/delta/events`;
